In [1]:
import numpy as np
qubit = np.array([1 / np.sqrt(2) + 0j, 1 / np.sqrt(2) + 0j]).reshape((2, 1))

In [2]:
qubit

array([[0.70710678+0.j],
       [0.70710678+0.j]])

In [3]:
# Z basis: |0> |1>
basis_0 = np.array([1 + 0j, 0 + 0j]).reshape((2, 1))
basis_1 = np.array([0 + 0j, 1 + 0j]).reshape((2, 1))

c0 = c1 = 1 / np.sqrt(2)

print(np.allclose(qubit, c0 * basis_0 + c1 * basis_1))

True


In [4]:
# probability for 1 and 0: its is about coeff in linear decomposition
p0 = np.conj(c0) * c0
p1 = np.conj(c1) * c1

print(np.allclose(p0, p1))
print(np.allclose(p0 + p1, 1.0))

True
True


In [6]:
p0, p1

(0.4999999999999999, 0.4999999999999999)

In [16]:
print(np.allclose(np.conj(qubit).T @ qubit, 1))

True


In [18]:
# X basis: |+> |->
plus = (basis_0 + basis_1) / np.sqrt(2)
minus = (basis_0 - basis_1) / np.sqrt(2)

In [19]:
# Y basis
R = (basis_0 + 1j * basis_1) / np.sqrt(2)
L = (basis_0 - 1j * basis_1) / np.sqrt(2)

In [22]:
np.allclose(np.conj(R).T @ L, 0)

True

# 1 qubit Gates

In [23]:
# Hadamar: 0 -> +
h = 1 / np.sqrt(2) * np.array([
    [1 + 0j, 1 + 0j],
    [1 + 0j, 0j - 1]
])

In [29]:
print(np.allclose(np.conj(h).T @ h, np.eye(2)))
print(np.allclose(basis_0.T @ h, plus))

True
True


In [31]:
# Pauly operators: measurments
pauli_x = np.array([[0 + 0j, 1 + 0j], [1 + 0j, 0 + 0j]])
pauli_y = np.array([[0 + 0j, 0 - 1j], [0 + 1j, 0 + 0j]])
pauli_z = np.array([[1 + 0j, 0 + 0j], [0 + 0j, 0j - 1]])

In [36]:
np.linalg.eig(pauli_z)

(array([ 1.+0.j, -1.+0.j]),
 array([[1.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j]]))

In [37]:
print(np.linalg.eig(pauli_x))
print(np.linalg.eig(pauli_y))

(array([ 1.+0.j, -1.+0.j]), array([[ 0.70710678-0.j,  0.70710678+0.j],
       [ 0.70710678+0.j, -0.70710678-0.j]]))
(array([ 1.+0.j, -1.+0.j]), array([[-0.        -0.70710678j,  0.70710678+0.j        ],
       [ 0.70710678+0.j        ,  0.        -0.70710678j]]))


In [38]:
# nothing can be said about M of sigma_z from +
print(plus.conj().T @ pauli_z @ plus)

[[-2.23711432e-17+0.j]]


In [39]:
print(plus.conj().T @ pauli_x @ plus)

[[1.+0.j]]


In [40]:
# projections
# Born rule
super_position = h @ basis_0
eigenvectors = np.linalg.eig(pauli_z)[1]

proj_0 = eigenvectors[0].reshape((-1, 1)) @ eigenvectors[0].reshape((1, -1))
proj_1 = eigenvectors[1].reshape((-1, 1)) @ eigenvectors[1].reshape((1, -1))

In [41]:
proj_0

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j]])

In [42]:
p_0 = super_position.conj().T @ proj_0 @ super_position
p_1 = super_position.conj().T @ proj_1 @ super_position

print(np.allclose(p_0 + p_1, 1.0))
print(np.allclose(p_0, 0.5))

True
True


In [43]:
# RY, RX, RZ gates

# RY
def ry(state, phi):
    return np.array([
        [np.cos(phi / 2), -np.sin(phi / 2)],
        [np.sin(phi / 2),  np.cos(phi / 2)]
    ]) @ state

In [44]:
# check for 0 with rotation by OY on pi/2 will lead to 1 (on OX) [as according to Bloh sphere]
def expval(state, op):  # measure state by pauli operator
    return state.conj().T @ op @ state

pauli_x = np.array([[0 + 0j, 1 + 0j], [1 + 0j, 0 + 0j]])

print(np.allclose(expval(ry(basis_0, np.pi / 2), pauli_x), 1.0))
print(np.allclose(expval(ry(basis_0, -np.pi / 2), pauli_x), -1.0))

True
True


In [47]:
random_state = np.array([0.42 + 0j, np.sqrt(1 - 0.42**2) + 0j]).reshape((2, 1))
state2pi = ry(random_state, 2 * np.pi)

print(np.allclose(expval(state2pi, pauli_x), expval(random_state, pauli_x)))

True


In [48]:
pauli_z = np.array([[1 + 0j, 0 + 0j], [0 + 0j, 0j - 1]])

print("Z:\n\t" + str(expval(random_state, pauli_z)) + "\n")
print("X:\n\t" + str(expval(random_state, pauli_x)) + "\n")

print("Z after RY:\n\t" + str(expval(ry(random_state, 2 * np.pi), pauli_z)) + "\n")
print("X after RY:\n\t" + str(expval(ry(random_state, 2 * np.pi), pauli_x)) + "\n")

Z:
	[[-0.6472+0.j]]

X:
	[[0.76232025+0.j]]

Z after RY:
	[[-0.6472+0.j]]

X after RY:
	[[0.76232025+0.j]]



In [49]:
# phase-shift gate aka RZ
def u1(state, phi):
    return np.array([[1, 0], [0, np.exp(1j * phi)]]) @ state

In [53]:
# equation for u2 = u1(lambda + phi) RZ(-lambda) RY(phi/2) RZ(lambda)
def rz(state, phi):
    return np.array([[np.exp(-1j * phi / 2), 0], [0, np.exp(1j * phi / 2)]]) @ state


def u2_direct(phi, l):
    return (
        1
        / np.sqrt(2)
        * np.array([[1, -np.exp(1j * l)], [np.exp(1j * phi), np.exp(1j * (phi + l))]])
    )

def u2_inferenced(phi, l):
    state = np.eye(2)  # hardcore for pruf
    return (
        u1(state, phi + l)
        @ rz(state, -l)
        @ ry(state, np.pi / 2)
        @ rz(state, l)
    )
print(np.allclose(u2_direct(np.pi / 6, np.pi / 3), u2_inferenced(np.pi / 6, np.pi / 3)))

True


In [58]:
# before using gate with phase change: it is better to use non |0>, or ur operator, e.g. u1, will not change.
# Use Hadamar for example: 0 -> +
# and ur operator will lead to smth
print(np.allclose(u1(basis_0, np.pi / 6), basis_0))

h = 1 / np.sqrt(2) * np.array([[1 + 0j, 1 + 0j], [1 + 0j, 0j - 1]])
print(np.allclose(u1(h @ basis_0, np.pi / 6), h @ basis_0))

True
False


In [59]:
identity_gate = np.eye(2, dtype=np.complex128)
print(identity_gate)

[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]


# Multi Qubit Gates

In [60]:
basis = basis_0
print(np.allclose(
    np.kron(h @ basis, basis), 
    np.kron(h, identity_gate) @ np.kron(basis, basis))
     )

True


In [62]:
# measure ZZ
np.kron(np.conj(basis).T, np.conj(basis).T) @ np.kron(pauli_z, pauli_z) @ np.kron(basis,basis)

array([[1.+0.j]])

In [66]:
# cnot gate (CX gate)
# note: pauly_x: 0 -> 1 and reverse
cnot = (1 + 0j) * np.array(
    [
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
    ]
)

print(np.allclose(cnot @ np.kron(basis, basis), np.kron(basis, basis)))
print(np.allclose(
    cnot @ np.kron(pauli_x @ basis, basis), 
    np.kron(pauli_x @ basis, pauli_x @ basis)
))

True
True


In [67]:
pauli_x @ basis, basis

(array([[0.+0.j],
        [1.+0.j]]),
 array([[1.+0.j],
        [0.+0.j]]))

# pennylane

In [73]:
np.kron(basis, basis), basis

(array([[1.+0.j],
        [0.+0.j],
        [0.+0.j],
        [0.+0.j]]),
 array([[1.+0.j],
        [0.+0.j]]))

In [75]:
np.kron(ry(np.eye(2), np.deg2rad(45)), np.eye(2, dtype=np.complex128))

array([[ 0.92387953+0.j,  0.        +0.j, -0.38268343+0.j,
        -0.        +0.j],
       [ 0.        +0.j,  0.92387953+0.j, -0.        +0.j,
        -0.38268343+0.j],
       [ 0.38268343+0.j,  0.        +0.j,  0.92387953+0.j,
         0.        +0.j],
       [ 0.        +0.j,  0.38268343+0.j,  0.        +0.j,
         0.92387953+0.j]])

In [77]:
ry(np.eye(2), np.deg2rad(45))

array([[ 0.92387953, -0.38268343],
       [ 0.38268343,  0.92387953]])

In [78]:
# Напишем функцию, которая поворачивает первый кубит на 45, после чего измеряет оба кубита по оси Z
state = np.kron(basis, basis)
op = np.kron(ry(np.eye(2), np.deg2rad(45)), np.eye(2, dtype=np.complex128))  # почему np.eye? когда скармливать нужно стейт
measure = np.kron(pauli_z, pauli_z)

print((op @ state).conj().T @ measure @ (op @ state))

[[0.70710678+0.j]]


In [80]:
import pennylane as qml
device = qml.device("default.qubit", 2)

@qml.qnode(device)
def test(angle):
    qml.RY(angle, wires=0)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))


print(test(np.deg2rad(45)))

0.7071067811865475
